# Problem from McDonald (2013)

## a.

To find the 3-year swap price, we equate the present value of receiving the forward prices ($20, $21, $22) to the present value of paying the fixed swap price (S) for 3 years, discounted at given interest rates.  Calculating this gives a 3-year swap price of approximately $20.95.

## b.

For the 2-year swap starting in one year, we equate the present value of receiving forward prices for year 2 and 3 ($21, $22) to the present value of paying a fixed swap price (S) for 2 years (settling at year 2 and 3), all discounted back to time 0.  Calculating this gives a swap price of approximately $21.48.

# Programming of Binomial pricing of European and American options

The code for this exercise is included in the attached notebook `main.ipynb`.

## a.

The `binomial_european_option_value` function calculates European option prices. It starts by computing parameters like time step, up/down factors, and risk-neutral probability, considering continuous dividends.  Then, it constructs a binomial tree of stock prices. At expiration, option payoffs are calculated.  Finally, it uses backward induction, discounting expected option values at each step to find the option price at time zero.  This process values the option by working backwards from expiration to the present.

In [33]:
import numpy as np


def binomial_european_option_value(
    S: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    q: float,
    N: int,
    option_type: str = "call",
) -> float:
    delta_t = T / N
    u = np.exp(sigma * np.sqrt(delta_t))
    d = 1 / u
    p = (np.exp((r - q) * delta_t) - d) / (u - d)

    # init stock price tree
    stock_prices = np.zeros((N + 1, N + 1))
    stock_prices[0, 0] = S

    for i in range(1, N + 1):
        for j in range(i + 1):
            stock_prices[i, j] = S * (u ** (j)) * (d ** (i - j))

    # init option value tree at expiration
    option_values = np.zeros((N + 1, N + 1))

    for j in range(N + 1):
        if option_type == "call":
            option_values[N, j] = max(0, stock_prices[N, j] - K)
        elif option_type == "put":
            option_values[N, j] = max(0, K - stock_prices[N, j])
        else:
            raise ValueError("Option type must be 'call' or 'put'")

    # Backward induction to find option value at time 0
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            option_values[i, j] = (
                p * option_values[i + 1, j + 1] + (1 - p) * option_values[i + 1, j]
            ) * np.exp(-r * delta_t)

    return option_values[0, 0]


S_european = 100  # Stock price
K_european = 100  # Strike price
T_european = 1  # Time to maturity (years)
r_european = 0.05  # Risk-free rate
sigma_european = 0.2  # Volatility
q_european = 0.02  # Dividend yield
N_european = 100  # Number of steps

european_call_value = binomial_european_option_value(
    S_european,
    K_european,
    T_european,
    r_european,
    sigma_european,
    q_european,
    N_european,
    option_type="call",
)
european_put_value = binomial_european_option_value(
    S_european,
    K_european,
    T_european,
    r_european,
    sigma_european,
    q_european,
    N_european,
    option_type="put",
)

# print(f"European Call Option Value: {european_call_value:.4f}\n")
# print(f"European Put Option Value: {european_put_value:.4f}")

## b.

The `binomial_american_option_value` function calculates American option prices. It largely follows the same initial steps as the European option function, including parameter calculation and stock price tree construction. The key difference is in the backward induction. For American options, at each step, the code considers both the continuation value (holding the option) and the intrinsic value (exercising immediately). The option value at each node is then the maximum of these two values. This accounts for the early exercise feature of American options, making sure the option holder always makes the optimal decision at each time step. The function also uses backward induction to arrive at the option price at time zero, but with the added consideration of early exercise at every step.

In [34]:
def binomial_american_option_value(
    S: float,
    K: float,
    T: float,
    r: float,
    sigma: float,
    q: float,
    N: int,
    option_type: str = "call",
) -> float:
    delta_t = T / N
    u = np.exp(sigma * np.sqrt(delta_t))
    d = 1 / u
    p = (np.exp((r - q) * delta_t) - d) / (u - d)

    # init stock price tree
    stock_prices = np.zeros((N + 1, N + 1))
    stock_prices[0, 0] = S

    for i in range(1, N + 1):
        for j in range(i + 1):
            stock_prices[i, j] = S * (u ** (j)) * (d ** (i - j))

    # init option value tree at expiration
    option_values = np.zeros((N + 1, N + 1))

    for j in range(N + 1):
        if option_type == "call":
            option_values[N, j] = max(0, stock_prices[N, j] - K)
        elif option_type == "put":
            option_values[N, j] = max(0, K - stock_prices[N, j])
        else:
            raise ValueError("Option type must be 'call' or 'put'")

    # Backward induction to find option value at time 0, considering early exercise
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            # continuation value
            continuation_value = (
                p * option_values[i + 1, j + 1] + (1 - p) * option_values[i + 1, j]
            ) * np.exp(-r * delta_t)

            # intrinsic value
            if option_type == "call":
                intrinsic_value = max(0, stock_prices[i, j] - K)
            elif option_type == "put":
                intrinsic_value = max(0, K - stock_prices[i, j])

            option_values[i, j] = max(intrinsic_value, continuation_value)

    return option_values[0, 0]


S_american = 100  # Stock price
K_american = 100  # Strike price
T_american = 1  # Time to maturity (years)
r_american = 0.05  # Risk-free rate
sigma_american = 0.2  # Volatility
q_american = 0.02  # Dividend yield
N_american = 100  # Number of steps

american_call_value = binomial_american_option_value(
    S_american,
    K_american,
    T_american,
    r_american,
    sigma_american,
    q_american,
    N_american,
    option_type="call",
)
american_put_value = binomial_american_option_value(
    S_american,
    K_american,
    T_american,
    r_american,
    sigma_american,
    q_american,
    N_american,
    option_type="put",
)

# print(f"American Call Option Value: {american_call_value:.4f}")
# print(f"American Put Option Value: {american_put_value:.4f}")

# Problem from other source

To value a 4-month European call option using a two-step binomial tree, we first calculate the key parameters for a 2-month time step using their formulas and get: $u \approx 1.1303$, $d \approx 0.8847$, and $p \approx 0.4899$, based on the given volatility, risk-free rate, and time step. Using these parameters, we construct a two-step binomial tree for the stock price starting at \$78 and progressing over two 2-month periods. At expiration (4 months), we determine the payoffs of the call option at each possible final stock price. By employing backward induction, we discount the expected option values at each node back to the initial time, resulting in an estimated value of approximately $4.66 for the 4-month European call option.

To hedge a short position of 1,000 of these call options (equivalent to 10 contracts) at the time of the trade, we calculate the delta of the option at time 0. Delta is found to be approximately 0.4989, representing the number of shares needed to hedge each option. Therefore, to hedge the sold options, the trader should buy approximately 499 shares of the underlying stock at the time of the trade. This long position in the stock offsets the risk associated with the short call option position.